# benign confounders labeling for hateful memes

## init

In [ ]:
# mount
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%capture
# hateful memes from MyDrive
!mkdir data
!unzip /content/drive/MyDrive/vilio/hateful_memes.zip -d data

In [ ]:
!git clone https://github.com/Noixas/Multimodal-NLP
!mkdir data/confounder
!cp Multimodal-NLP/dataset/upsampling_img_hash/image_confounders_id.json data/confounder
!cp Multimodal-NLP/dataset/upsampling_img_hash/image_confounders_id_dev_data.json data/confounder

In [ ]:
# modules
import glob
from itertools import chain
import json
import os
import signal
import sys
import time
from typing import Dict, List, Union
import warnings

import cv2
import dask.dataframe as dd
from dask.delayed import delayed
from IPython.display import clear_output
import matplotlib.pyplot as plt
# from matplotlib import image
import numpy as np
import pandas as pd
import pickle
import plotly.graph_objects as go
# from scipy.spatial import distance
from tqdm import tqdm, trange

%matplotlib inline
plt.style.use('dark_background')
warnings.simplefilter('ignore')

In [ ]:
# variables
data_root = "/content/data/hateful_memes"
img_path = "img"
jsons = []
txt_col, img_col, label_col = "text", "img", "label"
txt_cf_col, img_cf_col = "is_txt_confounder", "is_img_confounder"
txt_id_col, img_id_col = "txt_org_id", "img_org_id"
export_path = "/content/data/confounder"

## preprocessing

### jsonl

In [ ]:
def read_jsonls(path: str):
    """
    Read all jsonl files
    """

    def _read_jsonl(filename: str, usecols:List[str]=['id', 'img', label_col, 'text', "split"]):
        """
        Read one jsonl file w/ idenfitier
        """
        df = pd.read_json(filename, orient='records', lines=True)
        df["split"] = filename.split("/")[-1].split(".jsonl")[0]
        if label_col not in df.columns:
            df[label_col] = -1
        return df[usecols]

    jsons = glob.glob(f"{path}/*.jsonl")
    dd_list = [delayed(_read_jsonl)(json) for json in jsons]
    df = dd.from_delayed(dd_list).compute()
    return df

In [ ]:
df = read_jsonls(data_root)
df.head()

In [ ]:
# for compatibility with image filename
df["id"] = df["id"].astype('str').str.zfill(5)

## detecting benign confounder

### same-text-diff-image

In [ ]:
# flag duplicates
df_cnt = df[txt_col].value_counts().reset_index()
df_cnt.columns = [txt_col, "cnt"]
txt_dup = df_cnt.loc[df_cnt["cnt"]>=2, txt_col].tolist()
df[txt_cf_col] = 0
df.loc[df[txt_col].isin(txt_dup), txt_cf_col] = 1
df[txt_cf_col].value_counts()

### same-image-diff-text

In [ ]:
# load duplicates
with open(os.path.join(export_path, "image_confounders_id.json"), "r") as f:
    img_dup = json.load(f)
with open(os.path.join(export_path, "image_confounders_id_dev_data.json"), "r") as f:
    img_dup_dev = json.load(f)

In [ ]:
# make it compatible with filename
img_dup = [str(img).zfill(5) for img in img_dup]
img_dup_dev = [str(img).zfill(5) for img in img_dup_dev]
img_dup_all = img_dup+img_dup_dev

In [ ]:
# check duplicates
_idx, _idx2 = 1, 2

plt.figure()

#subplot(r,c) provide the no. of rows and columns
f, axarr = plt.subplots(1,2,figsize=(12,16))
for i,idx in enumerate([_idx,_idx2]):
    # use the created array to output your multiple images.
    img_id = img_dup[idx]
    img_path = f"img/{img_id}.png"
    label = df.loc[df["id"]==img_id, label_col].values[0]
    img = cv2.imread(os.path.join(data_root, img_path), cv2.IMREAD_GRAYSCALE)
    axarr[i].set_title(f"id={img_id}, label={label}")
    axarr[i].imshow(img, cmap="gray")

In [ ]:
# flag duplicates
df[img_cf_col] = 0
df.loc[df["id"].isin(img_dup_all), img_cf_col] = 1
df[img_cf_col].value_counts()

### explore confounder

In [ ]:
# text*image confounder cross aggregates
df.groupby([txt_cf_col, img_cf_col])["id"].nunique()

In [ ]:
'''
0-1, 1-0 should be image / text confounder, respectively
1-1 should be original (hateful?) image
'''

In [ ]:
# text*image confounder cross aggregates
df.groupby([label_col, txt_cf_col, img_cf_col])["id"].nunique()

In [ ]:
'''
text hateful: 1-1-[0,1]
text benign: 1-0-[0,1]
image hateful: 1-[0,1]-1
image benign: 0-[0,1]-1
-> once group of confounders is identified, can compare within group
'''

## grouping benign confounder

### text

In [ ]:
# select perfect duplicate
def duplicates_flagger(data:pd.DataFrame, data2:pd.DataFrame, mod_col:str=txt_col, label_col:str=label_col):
    def duplicate_flagger(ip1:Dict[str, Union[str, int]], ip2:Dict[str, Union[str, int]], mode:str="perfect"):
        """
        Compare txt/txt or img/img to check duplicate
        """
        if mode=="perfect":
            if ip1["data"]==ip2["data"]:
                out = ip2["id"]
            else:
                out = "Not hit"
        return out
    """
    Flagging duplicates out of single modality
    """
    if mod_col=="text":
        data_ip = data.copy()
        data_ip2 = data2[data2[label_col]==1].drop_duplicates(subset=mod_col)
    else:
        data_ip = data.copy()
    dup_ids = []
    for i, ip in tqdm(zip(data_ip["id"], data_ip[mod_col])):
        data_dict = {"id": i, "data": ip}
        cnt = 0
        for i2, ip2 in zip(data_ip2["id"], data_ip2[mod_col]):
            data_dict2 = {"id": i2, "data": ip2}
            dup_id = duplicate_flagger(data_dict, data_dict2)
            if dup_id!="Not hit":
                dup_ids.append(dup_id)
                break
            else:
                cnt += 1
                if cnt==len(data_ip2):
                    dup_ids.append(dup_id)
                else:
                    pass
    return dup_ids

In [ ]:
# df_txt_cf = df[df[txt_cf_col]==1].copy()
df_txt_cf = df[df[txt_cf_col]==1].sort_values(by=txt_col)
df_txt_cf[txt_id_col] = duplicates_flagger(df_txt_cf, df_txt_cf)

In [ ]:
df_txt_cf[txt_id_col].value_counts().head()

In [ ]:
fig = go.Figure()
df_plot = df_txt_cf[df_txt_cf[txt_id_col]!="Not hit"]
df_cnt = df_plot.loc[df_plot[txt_id_col]!="Not hit", txt_id_col].value_counts().reset_index()
fig.add_trace(go.Histogram(x=df_cnt[txt_id_col], name=f"id count", histnorm='probability', bingroup=1))
fig.show()

### image

#### annotation

In [ ]:
# img_dup_all = img_dup_all[:20]
# print(len(img_dup_all))

In [ ]:
def signal_handler(signal, frame):
    print('Aborted')
    sys.exit(0)

In [ ]:
signal.signal(signal.SIGINT, signal_handler)

num_img_org = len(img_dup_all)
cluster = 0
try:
    with open("/content/drive/MyDrive/annotation/hm_img_confounder.pickle", "rb") as f:
        clusters = pickle.load(f)
except Exception as e:
    clusters = []
if len(clusters)==0:
    cluster = 0
else:
    cluster = max(clusters)
done = len(clusters)
num_img = num_img_org-done
for idx in trange(num_img//2):
# for idx in trange(5):
    # check duplicates
    _idx, _idx2 = done+2*idx, done+2*idx+1

    if done:
        img_id = img_dup_all[idx]
        img_path = f"img/{img_id}.png"
        label = df.loc[df["id"]==img_id, label_col].values[0]
        img = cv2.imread(os.path.join(data_root, img_path), cv2.IMREAD_GRAYSCALE)

    plt.figure()

    # show images
    ## subplot(r,c) provide the no. of rows and columns
    f, axarr = plt.subplots(1,2,figsize=(12,16))
    for i,i2 in enumerate([_idx,_idx2]):
        ## use the created array to output your multiple images.
        img_id = img_dup_all[i2]
        img_path = f"img/{img_id}.png"
        label = df.loc[df["id"]==img_id, label_col].values[0]
        img = cv2.imread(os.path.join(data_root, img_path), cv2.IMREAD_GRAYSCALE)
        axarr[i].set_title(f"id={img_id}, label={label}")
        axarr[i].imshow(img, cmap="gray")
    plt.show()

    with open("/content/drive/MyDrive/annotation/hm_img_confounder.pickle", "wb") as f:
        pickle.dump(clusters, f)

    ## 1 if image is new, else 0
    left = input("LEFT image new?: ")
    right = input("RIGHT image new?: ")
    for result in [left, right]:
        if int(result):
            cluster += 1
        else:
            pass
        clusters.append(cluster)

    time.sleep(1)
    clear_output(wait=True)
with open("/content/drive/MyDrive/annotation/hm_img_confounder.pickle", "wb") as f:
    pickle.dump(clusters, f)

In [ ]:
# # check result
# arr_clusters = np.array(clusters)
# arr_img = np.array(img_dup_all)[:len(arr_clusters)]
# for idx in sorted(list(set(clusters)))[1000:1500]:
#     print(idx)
#     img_cluster = arr_img[arr_clusters==idx]
#     if len(img_cluster)==1:
#         continue
#     plt.figure()
#     f, axarr = plt.subplots(1,len(img_cluster),figsize=(4*len(img_cluster),6*len(img_cluster)))
#     for i,img_id in enumerate(img_cluster):
#         img_path = f"img/{img_id}.png"
#         label = df.loc[df["id"]==img_id, label_col].values[0]
#         img = cv2.imread(os.path.join(data_root, img_path), cv2.IMREAD_GRAYSCALE)
#         axarr[i].set_title(f"id={img_id}, label={label}, cluster={idx}")
#         axarr[i].imshow(img, cmap="gray")
#     plt.show()

In [ ]:
# # check result
# ## single-image cluster + representative image
# mode = "single"
# arr_clusters = np.array(clusters)
# arr_img = np.array(img_dup_all)[:len(arr_clusters)]
# for idx in sorted(list(set(clusters))):
#     img_cluster = arr_img[arr_clusters==idx]
#     if (mode=="single" and len(img_cluster)==1) or (mode!="single" and len(img_cluster)>1):
#         print(idx)
#         img_id = img_cluster[0]
#         img_path = f"img/{img_id}.png"
#         label = df.loc[df["id"]==img_id, label_col].values[0]
#         img = cv2.imread(os.path.join(data_root, img_path), cv2.IMREAD_GRAYSCALE)
#         plt.title(f"id={img_id}, label={label}, cluster={idx}, num_img={len(img_cluster)}")
#         plt.imshow(img, cmap="gray")
#         plt.show()
#     else:
#         pass

In [ ]:
# collect
arr_clusters,arr_clusters2 = np.array(clusters),np.array(clusters)
arr_clusters2[arr_clusters==98] = 106
arr_clusters2[arr_clusters==101] = 110
arr_clusters2[arr_clusters==495] = 496
clusters = arr_clusters2.tolist()
with open("/content/drive/MyDrive/annotation/hm_img_confounder.pickle", "wb") as f:
    pickle.dump(clusters, f)

In [ ]:
# # check result
# ## single-image cluster + representative image
# mode = "multiple"
# arr_clusters = np.array(clusters)
# arr_img = np.array(img_dup_all)[:len(arr_clusters)]
# for idx in sorted(list(set(clusters))):
#     img_cluster = arr_img[arr_clusters==idx]
#     if (mode=="single" and len(img_cluster)==1) or (mode!="single" and len(img_cluster)>1):
#         print(idx)
#         img_id = img_cluster[0]
#         img_path = f"img/{img_id}.png"
#         label = df.loc[df["id"]==img_id, label_col].values[0]
#         img = cv2.imread(os.path.join(data_root, img_path), cv2.IMREAD_GRAYSCALE)
#         plt.title(f"id={img_id}, label={label}, cluster={idx}, num_img={len(img_cluster)}")
#         plt.imshow(img, cmap="gray")
#         plt.show()
#     else:
#         pass

In [ ]:
# collect
arr_clusters,arr_clusters2 = np.array(clusters),np.array(clusters)
arr_clusters2[arr_clusters==620] = 617
clusters = arr_clusters2.tolist()
with open("/content/drive/MyDrive/annotation/hm_img_confounder.pickle", "wb") as f:
    pickle.dump(clusters, f)

In [ ]:
# add cluster for last image
## check if it's identical to -2
img_id = img_dup_all[-1]
img_path = f"img/{img_id}.png"
label = df.loc[df["id"]==img_id, label_col].values[0]
img = cv2.imread(os.path.join(data_root, img_path), cv2.IMREAD_GRAYSCALE)
plt.imshow(img, cmap="gray")

In [ ]:
# add cluster for last image
clusters.append(clusters[-1])
with open("/content/drive/MyDrive/annotation/hm_img_confounder.pickle", "wb") as f:
    pickle.dump(clusters, f)

In [ ]:
len(img_dup_all),len(clusters)

#### label to dataframe

In [ ]:
with open("/content/drive/MyDrive/annotation/hm_img_confounder.pickle", "rb") as f:
    clusters = pickle.load(f)

In [ ]:
# prep image clustered df
df_img_cf = pd.DataFrame([img_dup_all, clusters]).T
df_img_cf.columns = ["img_id", "img_cluster"]
# remove single-image cluster
df_img_cf_cnt = df_img_cf.groupby("img_cluster").count().reset_index()
single_cluster = df_img_cf_cnt.loc[df_img_cf_cnt["img_id"]==1, "img_cluster"].tolist()
print(len(df_img_cf))
df_img_cf = df_img_cf[~df_img_cf["img_cluster"].isin(single_cluster)]
print(len(df_img_cf))
# merge label
df_img_cf = df_img_cf.merge(df[["id", "label"]].drop_duplicates().set_index("id"), left_on="img_id", right_index=True)
df_img_cf_cnt = df_img_cf.groupby("img_cluster")["label"].sum().reset_index()
all_negatives = df_img_cf_cnt.loc[df_img_cf_cnt["label"]==0, "img_cluster"].tolist()
print(len(df_img_cf))
df_img_cf = df_img_cf[~df_img_cf["img_cluster"].isin(all_negatives)]
print(len(df_img_cf))
# add id by labels
## id of positive image, youngest if multiple
df_img_cf_pos = df_img_cf[df_img_cf["label"]==1].groupby("img_cluster")["img_id"].min().reset_index()
df_img_cf = df_img_cf.merge(df_img_cf_pos[["img_cluster", "img_id"]].rename({"img_id": img_id_col}, axis=1).set_index("img_cluster"), left_on="img_cluster", right_index=True)
df_img_cf.head()
# df_img_cf_pos.head()
# df_img_cf_cnt.head()

### group for evaluation

In [ ]:
len(df), len(df_img_cf), len(df_txt_cf)

In [ ]:
print(df.columns)
print(df_img_cf.columns)
print(df_txt_cf.columns)

In [ ]:
df_merged = df[["id", "split", "label"]].merge(df_img_cf[["img_id", "img_org_id"]].set_index("img_id"),left_on="id", right_index=True, how="left").fillna("No Confounder")
df_merged = df_merged.merge(df_txt_cf[["id", "txt_org_id"]].set_index("id"),left_on="id", right_index=True, how="left").fillna("No Confounder")

In [ ]:
df_merged.groupby(['split', 'label'])["img_org_id"].nunique()

In [ ]:
df_merged.groupby(['split', 'label'])["txt_org_id"].nunique()

In [ ]:
df_merged.to_parquet("/content/drive/MyDrive/annotation/confounders.parquet", index=False)

# EOS